# Normalization

## Rules implemented:
* Іі Ѣѣ Ѳѳ Ѵѵ --> Ии Ее Фф Ии
* jeja
* Final tverdyj znak
* prefixes before z

## todo


* ago, jago
* yja, ija
* o after palatals
* e oborotnoe before i

## issues
* not checking for stacked prefixes, assume prefix is at beginning of the word


благой
всеблагой
нагой
полунагой
преблагой


In [55]:
import re

def replace_letters(inputtext):
    # Replace Іі Ѣѣ Ѳѳ Ѵѵ
    oldstring = "ІіѢѣѲѳѴѵ"
    newstring = "ИиЕеФфИи"
    translationtable = str.maketrans(oldstring, newstring)
    return inputtext.translate(translationtable)

    
def eja(inputtext):    
    # jeja|ея|ЕЯ
    eja = re.sub(r'\bЕЯ\b', 'ЕЕ', re.sub(r'\bея\b', 'ее', re.sub(r'\bЕя\b', 'Ее' ,inputtext)))
    return eja
    
    
def tverdyj(inputtext):
    # Final tverdyj znak
    tverdyj = re.sub(r'\B[ъЪ]\b', '', inputtext)
    return tverdyj
    

def prefixZ(inputtext):
    # prefixes before z
    prefixZ = re.sub(r'\b(БЕ|В|ВО|И|НИ|РА|РО|ЧЕРЕ|ЧРЕ)З([ПФТСКЦШЩЧ])', r'\1с\2', \
                            re.sub(r'\b(Бе|В|Во|И|Ни|Ра|Ро|Чере|Чре)з([пфтскцшщч])', r'\1с\2' ,\
                            re.sub(r'\b(бе|в|во|и|ни|ра|ро|чере|чре)з([пфтскцшщч])', r'\1с\2', inputtext)))
    return prefixZ
    
def ago(inputtext):
    exceptions = ('благо', 'всеблаго', 'наго', 'полунаго', 'преблаго')
    if re.search(r'[яа]го\b', inputtext) and inputtext not in exceptions:
        return re.sub(r'[яа]го\b', 'ого', inputtext)
    else:
        return inputtext
        
    

def normalize(inputtext):
    return ago(prefixZ(tverdyj(eja(replace_letters(re.sub("\s+$","", inputtext))))))
    
    
inputstring = "всеблаго краснаго ІіѢѣѲѳѴѵ ея Ея бея цвѣтовъ безполезный, съ отраженіями звѣздъ въ спящей водѣ рѣки возсоденинение."


def create_token(inputtext):
    return {'t':inputtext, 'n':normalize(inputtext)}

[create_token(token) for token in re.findall('\w+\s*|\W+',inputstring)]

[{'n': 'всеблаго', 't': 'всеблаго '},
 {'n': 'красного', 't': 'краснаго '},
 {'n': 'ИиЕеФфИи', 't': 'ІіѢѣѲѳѴѵ '},
 {'n': 'ее', 't': 'ея '},
 {'n': 'Ее', 't': 'Ея '},
 {'n': 'бея', 't': 'бея '},
 {'n': 'цветов', 't': 'цвѣтовъ '},
 {'n': 'бесполезный', 't': 'безполезный'},
 {'n': ',', 't': ', '},
 {'n': 'с', 't': 'съ '},
 {'n': 'отражениями', 't': 'отраженіями '},
 {'n': 'звезд', 't': 'звѣздъ '},
 {'n': 'в', 't': 'въ '},
 {'n': 'спящей', 't': 'спящей '},
 {'n': 'воде', 't': 'водѣ '},
 {'n': 'реки', 't': 'рѣки '},
 {'n': 'воссоденинение', 't': 'возсоденинение'},
 {'n': '.', 't': '.'}]

### Resources:
https://ru.wikisource.org/wiki/Раздѣлъ:_Дореформенная_орѳографія